In [13]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [14]:
application_df.SPECIAL_CONSIDERATIONS.value_counts()

N    34272
Y       27
Name: SPECIAL_CONSIDERATIONS, dtype: int64

In [15]:
application_df.STATUS.value_counts()

1    34294
0        5
Name: STATUS, dtype: int64

In [16]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop('EIN',1)
application_df = application_df.drop('NAME',1)
application_df = application_df.drop('STATUS',1)
app_type_counts = application_df.APPLICATION_TYPE.value_counts()
app_type_counts
replace_application = list(app_type_counts[app_type_counts < 300].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")

app_class_counts = application_df.CLASSIFICATION.value_counts()
replace_class = list(app_class_counts[app_class_counts < 1800].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")


In [17]:
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()

In [18]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [19]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df,left_index=True, right_index=True)
application_df = application_df.drop(application_cat,1)


In [20]:
# Split our preprocessed data into our features and target arrays
y = application_df.IS_SUCCESSFUL.values
X = application_df.drop(columns=["IS_SUCCESSFUL", "SPECIAL_CONSIDERATIONS_Y",  "SPECIAL_CONSIDERATIONS_N"]).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)


In [21]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [22]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  100
hidden_nodes_layer2 = 60

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="sigmoid")
)


# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="sigmoid"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))


# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 100)               4100      
_________________________________________________________________
dense_9 (Dense)              (None, 60)                6060      
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 61        
Total params: 10,221
Trainable params: 10,221
Non-trainable params: 0
_________________________________________________________________


In [23]:
import os
from tensorflow.keras.callbacks import ModelCheckpoint
# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

nn.save("AlphabetSoupCharity_Optimization.h5")
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=1000)
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=75) 
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=10)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/75
804/804 [==============================] - 1s 785us/step - loss: 0.6164 - accuracy: 0.6757
Epoch 2/75
804/804 [==============================] - 1s 717us/step - loss: 0.5739 - accuracy: 0.7210
Epoch 3/75
804/804 [==============================] - 1s 699us/step - loss: 0.5736 - accuracy: 0.7193
Epoch 4/75
804/804 [==============================] - 1s 708us/step - loss: 0.5659 - accuracy: 0.7227
Epoch 5/75
804/804 [==============================] - 1s 695us/step - loss: 0.5619 - accuracy: 0.7268
Epoch 6/75
804/804 [==============================] - 1s 745us/step - loss: 0.5595 - accuracy: 0.7292
Epoch 7/75
804/804 [==============================] - 1s 736us/step - loss: 0.5538 - accuracy: 0.7330
Epoch 8/75
804/804 [==============================] - 1s 704us/step - loss: 0.5598 - accuracy: 0.7240
Epoch 9/75
804/804 [==============================] - 1s 710us/step - loss: 0.5471 - accuracy: 0.7349
Epoch 10/75
804/804 [==============================] - 1s 710us/step - loss: 0.551

In [24]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  200
hidden_nodes_layer2 = 100
hidden_nodes_layer3 = 50
hidden_nodes_layer4 = 5
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)


# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="relu"))
# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))


# Check the structure of the model
nn.summary()



Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_11 (Dense)             (None, 200)               8200      
_________________________________________________________________
dense_12 (Dense)             (None, 100)               20100     
_________________________________________________________________
dense_13 (Dense)             (None, 50)                5050      
_________________________________________________________________
dense_14 (Dense)             (None, 5)                 255       
_________________________________________________________________
dense_15 (Dense)             (None, 1)                 6         
Total params: 33,611
Trainable params: 33,611
Non-trainable params: 0
_________________________________________________________________


In [25]:

# Define the checkpoint path and filenames

# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=1000)
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=90) 
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=10)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/90
804/804 [==============================] - 1s 923us/step - loss: 0.5883 - accuracy: 0.7112
Epoch 2/90
804/804 [==============================] - 1s 975us/step - loss: 0.5574 - accuracy: 0.7257
Epoch 3/90
804/804 [==============================] - 1s 908us/step - loss: 0.5503 - accuracy: 0.7342
Epoch 4/90
804/804 [==============================] - 1s 907us/step - loss: 0.5467 - accuracy: 0.7365
Epoch 5/90
804/804 [==============================] - 1s 914us/step - loss: 0.5428 - accuracy: 0.7389
Epoch 6/90
804/804 [==============================] - 1s 916us/step - loss: 0.5488 - accuracy: 0.7329
Epoch 7/90
804/804 [==============================] - 1s 913us/step - loss: 0.5485 - accuracy: 0.7330
Epoch 8/90
804/804 [==============================] - 1s 935us/step - loss: 0.5452 - accuracy: 0.7388
Epoch 9/90
804/804 [==============================] - 1s 909us/step - loss: 0.5472 - accuracy: 0.7348
Epoch 10/90
804/804 [==============================] - 1s 911us/step - loss: 0.540

804/804 [==============================] - 1s 919us/step - loss: 0.5321 - accuracy: 0.7417
Epoch 81/90
804/804 [==============================] - 1s 917us/step - loss: 0.5396 - accuracy: 0.7343
Epoch 82/90
804/804 [==============================] - 1s 944us/step - loss: 0.5280 - accuracy: 0.7453
Epoch 83/90
804/804 [==============================] - 1s 928us/step - loss: 0.5272 - accuracy: 0.7473
Epoch 84/90
804/804 [==============================] - 1s 929us/step - loss: 0.5313 - accuracy: 0.7407
Epoch 85/90
804/804 [==============================] - 1s 933us/step - loss: 0.5496 - accuracy: 0.7371
Epoch 86/90
804/804 [==============================] - 1s 940us/step - loss: 0.5280 - accuracy: 0.7464
Epoch 87/90
804/804 [==============================] - 1s 919us/step - loss: 0.5321 - accuracy: 0.7420
Epoch 88/90
804/804 [==============================] - 1s 977us/step - loss: 0.5299 - accuracy: 0.7433
Epoch 89/90
804/804 [==============================] - 1s 927us/step - loss: 0.5281 -

In [26]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  150
hidden_nodes_layer2 = 100
hidden_nodes_layer3 = 50
hidden_nodes_layer4 = 5
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="tanh")
)


# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="tanh"))
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="tanh"))
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="tanh"))
# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="tanh"))


# Check the structure of the model
nn.summary()


Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 150)               6150      
_________________________________________________________________
dense_17 (Dense)             (None, 100)               15100     
_________________________________________________________________
dense_18 (Dense)             (None, 50)                5050      
_________________________________________________________________
dense_19 (Dense)             (None, 5)                 255       
_________________________________________________________________
dense_20 (Dense)             (None, 1)                 6         
Total params: 26,561
Trainable params: 26,561
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Define the checkpoint path and filenames

# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=1000)
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=90) 
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=20)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/90
804/804 [==============================] - 1s 864us/step - loss: 2.5070 - accuracy: 0.6797
Epoch 2/90
804/804 [==============================] - 1s 919us/step - loss: 1.8208 - accuracy: 0.6894
Epoch 3/90
804/804 [==============================] - 1s 898us/step - loss: 1.4175 - accuracy: 0.7137
Epoch 4/90
400/804 [=============>................] - ETA: 0s - loss: 1.3095 - accuracy: 0.7195